<a href="https://colab.research.google.com/github/heliton1986/Tutorial_PySpark/blob/main/Tutorial_6_Pyspark_With_Python_Introduction_To_Pyspark_Mlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Examples Of Pyspark ML** 

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 62.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a41dbd77a8218c0f104294e193f67bf692bc48f20e94da8667453ac871175e23
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
# Importar da biblioteca pyspark.sql SparkSession
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Pyspark Mlib").getOrCreate()

In [5]:
spark

In [6]:
# Lendo o dataset
training = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/PySpark Tutorial/tutorial_Pyspark6.csv', 
                          header=True, sep=';', inferSchema=True)

In [7]:
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
# Mostrar colunas
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [ ]:
# ['Age', 'Experience'] ----> new feature ----> independent feature

In [11]:
# Importar VectorAssembler da biblioteca pyspark.ml.feature
from pyspark.ml.feature import VectorAssembler

In [20]:
# Feature que crie uma coluna com os valores das colunas 'Age ' e 'Experience'
featureAssembler = VectorAssembler(inputCols=['Age', 'Experience'], outputCol='Independent Features')

In [21]:
# Variável de saída que transforme o dataset com a coluna criada
output= featureAssembler.transform(training)

In [22]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [28]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [29]:
# Variavel que mostre as colunas 'Independent Features' e 'Salary'
finalized_data = output.select('Independent Features', 'Salary')

In [30]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [31]:
from pyspark.ml.regression import LinearRegression

In [41]:
# Criar variavel de treino, teste
# Train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [42]:
# Coeficientes
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [43]:
# Intecepts
regressor.intercept

16079.13669064716

In [44]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [45]:
pred_results.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|17214.090796328448|
+--------------------+------+------------------+



In [46]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(785.909203671552, 617653.276415653)